In [ ]:
import os
import re
import requests
import html
from bs4 import BeautifulSoup

links = []

#краулер ищет ссылки на статьи по страницам (171) 
def articles_links():
    for i in range(10):   #их тут всего 171, но я не буду брать все, слишком тяжело и все падает
        try:
            url = 'http://pav-edin23.ru/category/obshhestvo/page/' + str(i) + '/'
            r = requests.get(url)
            text = r.text
            soup = BeautifulSoup(text, 'lxml')
        except:
            print('Error at: ', url)
        divs = soup.findAll('div', {'class': 'entry-thumb'})
        for div in divs:
            for anchor in div.find_all('a'):
                a = anchor.get('href')
                links.append(a)

#Создаем каталог plain + mystem-xml
def katalog():
    path = os.getcwd()
    h_row = ['path', 'author', 'header', 'created', 'sphere', 'topic', 'style', 'audience_age', 'audience_level', 
             'audience_size', 'source', 'publication', 'publ_year', 'medium', 'country', 'region', 'language']
    if not os.path.exists(path + '\\newspaper'):
                os.makedirs(path + '\\newspaper')
    meta_name = path + '\\newspaper\\metadata.csv'
    with open( meta_name,'w', encoding='utf-8') as f:
        header_row = '\t'.join(h_row)
        f.write(header_row)
        f.write('\n')                                                         #создали заголовки мета-таблицы
       
    regTag = re.compile('<.*?>', re.DOTALL)                                     # это рег. выражение находит все тэги
    regScript = re.compile('<script>.*?</script>', re.DOTALL)                   # все скрипты
    regComment = re.compile('<!--.*?-->', re.DOTALL)                            # все комментарии
    regNum = re.compile(r'[-+]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][-+]?\d+)?')       #все цифровые значения
    regLang = re.compile(r'[^ЁА-Яёа-я ]')                                       #нерусские слова     
    for link in links:
        r = requests.get(link)
        text = r.text
        soup = BeautifulSoup(text, 'lxml')
        divs = soup.findAll('div', {'class': 'entry entry-content'})
        for div in divs:
            p = div.findAll('p')
            clean_p = regScript.sub("", str(p))
            clean_p = regComment.sub("", clean_p)
            clean_p = regTag.sub("", clean_p)
            clean_p = html.unescape(clean_p)
            clean_p = regNum.sub("",clean_p)
            clean_p = regLang.sub("", clean_p)                                  #Это отчищенный и готовый текст статьи
            title = re.findall(r'<h1 class="entry-title">(.*?)</h1>', text)
            data = re.compile(r'[\d{4}/\d{2}/\d{2}]')
            data = data.findall(link)
            year = ''.join(data[5:9])
            month = ''.join(data[10:12])
            day = ''.join(data[13:15])
            name = link.split('/')[-2]
            
            if not os.path.exists(path + '\\newspaper' + '\\plain'):
                os.makedirs(path + '\\newspaper' + '\\plain')
            if not os.path.exists(path + '\\newspaper' + '\\plain\\' + year ):
                os.makedirs(path + '\\newspaper' + '\\plain\\'  + year)
            if not os.path.exists(path + '\\newspaper'+ '\\plain\\' + year + '\\' + month):
                os.makedirs(path + '\\newspaper'+ '\\plain\\' + year + '\\' + month)
            if not os.path.exists(path + '\\newspaper'+ '\\plain\\' + year + '\\' + month + '\\' + day):
                os.makedirs(path + '\\newspaper'+ '\\plain\\' + year + '\\' + month+ '\\' + day)
            txt_name = path + '\\newspaper\\plain\\' + year + '\\' + month+ '\\' + day + '\\' + name + '.txt'
            
            path_ = txt_name
            author = "none"
            header = title[0]
            created = day + '.' + month + '.' + year
            sphere = "публицистика"
            topic = 'общество'
            style = "нейтральный"
            audience_age = "н-возраст"
            audience_level = 'н-уровень'
            audience_size = 'районная'
            source = link
            publication = 'Единство'
            publ_year = year
            medium = "газета"
            country =  "Россия"
            region = 'Павловский район'
            language = "ru"
            
            with open(txt_name,'w', encoding='utf-8') as f:               
                au = '@au ' + author + '\n'                      #это все потом очень сильно мешается
                ti = '@ti ' + header + '\n'                     #но нет времени их убирать
                da = '@da ' + created + '\n'
                top = '@topic ' + topic + '\n'
                ur = '@url' + link + '\n'
                f.write(au)
                f.write(ti)
                f.write(da)
                f.write(top)
                f.write(ur)
                f.write(clean_p)                                      #создали необработанные тексты статей
                
                
            row = [path_, author, header, created, sphere, topic, style, audience_age, audience_level, audience_size, source,
                   publication, publ_year, medium, country, region, language]
            new_row = '\t'.join(row)
            with open( meta_name,'a', encoding='utf-8') as f:
                f.write(new_row)
                f.write('\n')
                
            if not os.path.exists(path + '\\newspaper\\mystem-xml'):
                os.makedirs(path + '\\newspaper\\mystem-xml')
            if not os.path.exists(path + '\\newspaper\\mystem-xml\\' + year ):
                os.makedirs(path + '\\newspaper\\mystem-xml\\'  + year)
            if not os.path.exists(path + '\\newspaper\\mystem-xml\\' + year + '\\' + month):
                os.makedirs(path + '\\newspaper\\mystem-xml\\' + year + '\\' + month)
            if not os.path.exists(path + '\\newspaper\\mystem-xml\\' + year + '\\' + month + '\\' + day):
                os.makedirs(path + '\\newspaper\\mystem-xml\\' + year + '\\' + month+ '\\' + day)  
            xml_name = path + '\\newspaper\\mystem-xml\\' + year + '\\' + month+ '\\' + day + '\\' + name + '.xml'
            
            #создаем обработанные тексты статей
            os.system(r'C:\\Users\\User\\Desktop\\mystem.exe -cdgin --format xml --eng-gr {} {}'.format(txt_name,xml_name))
            
            if not os.path.exists(path + '\\newspaper\\mystem-plain'):
                os.makedirs(path + '\\newspaper\\mystem-plain')
            if not os.path.exists(path + '\\newspaper\\mystem-plain\\' + year ):
                os.makedirs(path + '\\newspaper\\mystem-plain\\'  + year)
            if not os.path.exists(path + '\\newspaper\\mystem-plain\\' + year + '\\' + month):
                os.makedirs(path + '\\newspaper\\mystem-plain\\' + year + '\\' + month)
            if not os.path.exists(path + '\\newspaper\\mystem-plain\\' + year + '\\' + month + '\\' + day):
                os.makedirs(path + '\\newspaper\\mystem-plain\\' + year + '\\' + month+ '\\' + day)  
            plain_name = path + '\\newspaper\\mystem-plain\\' + year + '\\' + month+ '\\' + day + '\\' + name + '.txt'
            
            os.system(r'C:\\Users\\User\\Desktop\\mystem.exe -cdgin  --eng-gr {} {}'.format(txt_name,plain_name))
            

def main():
    articles_links()
    katalog()
main()

